# Inference (on real data)

In [ ]:
from IPython.display import HTML      # For animation.
from matplotlib import animation      # For animation.
from matplotlib import pyplot as plt  # For plotting.
import numpy as np                    # For solving ODE.
from scipy.integrate import odeint    # For solving ODE.
import cma                            # For optimization.

## Reading data 

In [ ]:
# Read data from the file.
data = np.loadtxt('06_data.txt')

# Extract time points (first column).
data_t = data[:, 0]

# Extract prey and predator (third and second column, respectively).
data_y = np.concatenate((data[:, 2:3], data[:, 1:2]), axis=1)

# Initial condition.
y0 = data_y[0]

In [ ]:
# Time-population plot.
# Show only first set of initial conditions.
plt.plot(data_t, data_y[:, 0], '--o', label='prey')
plt.plot(data_t, data_y[:, 1], '--o', label='predator')
plt.xlabel('time')
plt.ylabel('population')
plt.locator_params(axis='x', nbins=11)
plt.grid(linestyle=':')
plt.legend()
plt.show()

## Optimization (finding optimal parameters) 

In [ ]:
def evaluate(params):
    """Given some parameters, calculate how much does
    the Lotka-Vortella result deviate from data."""
    a, b, c, d = params
    F = lambda y, t: [
        y[0] * (a - b * y[1]),   # How population 0 changes.
        y[1] * (-c + d * y[0]),  # How population 1 changes.
    ]

    # Evaluate at the same points the data is available at.
    y = odeint(F, y0, data_t)

    # We define deviation as the sum of squares of differences (Frobenius norm).
    result = np.linalg.norm(y - data_y, 'fro')

    return result

In [ ]:
initial_guess = [0.1, 0.05, 0.1, 0.05]  # Initial guess of (a, b, c, d).
lower_bounds = [0.1, 0.001, 0.1, 0.001]
upper_bounds = [0.9, 0.1, 0.9, 0.1]

def constrained_evaluation(params):
    """Wrap `evaluate` function with a penalization that
    keeps the parameters in specific ranges."""
    penalty = 0
    for lb, v, ub in zip(lower_bounds, params, upper_bounds):
        if v < lb:
            penalty += (lb - v) ** 2
        elif v > ub:
            penalty += (v - ub) ** 2
    return 1e6 * penalty + evaluate(params)

es = cma.CMAEvolutionStrategy(initial_guess, 0.5, {'popsize': 64})
cma_result = es.optimize(constrained_evaluation)

print("\nOptimization done! Best parameters:", cma_result.best.get()[0])

In [ ]:
# Best parameter set is:
cma_result.best.get()[0]

## Visualization of the optimal solution

In [ ]:
# Extract parameters as a_, b_, c_ and d_.
a_, b_, c_, d_ = cma_result.best.get()[0]
F_ = lambda y, t: [
    y[0] * (a_ - b_ * y[1]),   # How population 0 changes.
    y[1] * (-c_ + d_ * y[0]),  # How population 1 changes.
]

t = np.linspace(data_t[0], data_t[-1], 100)
y = odeint(F_, y0, t)

In [ ]:
# Time-population plot.
plt.plot(data_t, data_y[:, 0], '--o', color='tab:blue', label='prey (data)')
plt.plot(data_t, data_y[:, 1], '--o', color='tab:orange', label='predator (data)')
plt.plot(t, y[:, 0], color='tab:blue', label='prey')
plt.plot(t, y[:, 1], color='tab:orange', label='predator')
plt.xlabel('time')
plt.ylabel('population')
plt.locator_params(axis='x', nbins=11)
plt.grid(linestyle=':')
plt.legend()
plt.show()